# Create AI API client in python

In [ ]:
root_path = "/Users/I559573/Library/CloudStorage/OneDrive-SAPSE/Documents/D2V2.0_PREP/btp-ai-core-bootcamp/src\
/ai-models/defect-detection"

In [ ]:
import sys, os
import json
import requests
import base64
import time
import yaml
from IPython.display import clear_output
from pprint import pprint
import ast
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import cv2

from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.status import Status
from ai_api_client_sdk.models.target_status import TargetStatus
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

In [ ]:
resource_group = "defect-det"  # Must be created before

aic_service_key = root_path + "/files/aic_service_key.json" # ENSURE YOU HAVE THE FILE PLACED CORRECTLY
with open(aic_service_key) as ask:
    aic_s_k = json.load(ask)

# NO CHANGES REQUIRED BELOW
#
ai_api_v2_client = AIAPIV2Client(
    base_url=aic_s_k["serviceurls"]["AI_API_URL"] + "/v2/lm",
    auth_url=aic_s_k["url"] + "/oauth/token",
    client_id=aic_s_k['clientid'],
    client_secret=aic_s_k['clientsecret'],
    resource_group=resource_group)

# Train execution of ML Model

Create a training configuration

In [ ]:
input_artifact_name = training_workflow['spec']['templates'][0]['inputs']['artifacts'][0]['name']
executable_name = training_workflow['metadata']['name']

artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": artifact_resp.id
}

train_configuration = {
    "name": "dev-tutorial-training-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

# store the configuration response to access the id to create an execution
train_config_resp = ai_api_v2_client.configuration.create(**train_configuration)
pprint(vars(train_config_resp))

assert train_config_resp.message == 'Configuration created'

print("Configuration created for running the training")

Create a training execution

In [ ]:
execution_resp = ai_api_v2_client.execution.create(train_config_resp.id)
pprint(vars(execution_resp))

Observe the training status

In [ ]:
status = None
while status != Status.COMPLETED and status != Status.DEAD:
    # Sleep for 5 secs to avoid overwhelming the API with requests
    time.sleep(5)
    # Clear outputs to reduce clutter
    clear_output(wait=True)

    execution = ai_api_v2_client.execution.get(execution_resp.id)
    status = execution.status
    print('...... execution status ......', flush=True)
    print(f"Training status: {execution.status}")
    pprint(f"Training status details: {execution.status_details}")

if execution.status == Status.COMPLETED:
    print(f"Training complete for execution [{execution_resp.id}]!")
    output_artifact = execution.output_artifacts[0]
    output = {
        "id": output_artifact.id,
        "name": output_artifact.name,
        "url": output_artifact.url
    }
    with open('training_output.json', 'w') as fp: #Save the reference to the model stored in S3
        json.dump(output, fp)

# Metrics and performance

In [ ]:
filter_string = "executionId eq '" + execution_resp.id + "'"
metric_resp = ai_api_v2_client.metrics.query(execution_ids=execution_resp.id)

for m in metric_resp.resources:
    for metric in m.metrics:
        print(metric.name)
        print(metric.value)

In [ ]:
all_metrics = []
for m in metric_resp.resources:
    for custom_info in m.custom_info:
        #print(custom_info.name)
        #print(custom_info.value)
        all_metrics.append(custom_info.value)

In [ ]:
training_metrics = ast.literal_eval(all_metrics[0])
fig, axs = plt.subplots(1, 2, figsize=(20,5))

a = ast.literal_eval(training_metrics[0].get("loss"))
b = ast.literal_eval(training_metrics[1].get("val_loss"))
c = ast.literal_eval(training_metrics[2].get("iou"))
d = ast.literal_eval(training_metrics[3].get("val_iou"))

axs[0].plot(a)
axs[0].plot(b)
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Validation'])

axs[1].plot(c)
axs[1].plot(d)
axs[1].title.set_text('Training IoU vs Validation IoU')
axs[1].legend(['Train', 'Validation'])

a = plt.setp(axs[0], xlabel='Epoch')
a = plt.setp(axs[0], ylabel='Loss')
a = plt.setp(axs[1], xlabel='Epoch')
a = plt.setp(axs[1], ylabel='IoU')

# Deploy ML Model

In [ ]:
serving_workflow_file = root_path + "/workflows/serving_workflow_tutorial.yaml"
with open(serving_workflow_file) as swf:
    serving_workflow = yaml.safe_load(swf)

scenario_id = serving_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
input_artifact_name = serving_workflow['spec']['inputs']['artifacts'][0]['name']
executable_name = serving_workflow['metadata']['name']

training_output = 'training_output.json'
with open(training_output) as to:
    serving_input = json.load(to)

In [ ]:
artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": serving_input["id"]
}

serve_configuration = {
    "name": "dev-tutorial-serving-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

serve_config_resp = ai_api_v2_client.configuration.create(**serve_configuration)

assert serve_config_resp.message == 'Configuration created'

pprint(vars(serve_config_resp))
print("configuration for serving the model created")

In [ ]:
deployment_resp = ai_api_v2_client.deployment.create(serve_config_resp.id)
pprint(vars(deployment_resp))

In [ ]:
# Poll deployment status
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    #deployment_resp_id = "d82a366155be8696"
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    #deployment = ai_api_v2_client.deployment.get(deployment_resp_id)
    status = deployment.status
    print('...... deployment status ......', flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")
        #print(f"Deployment with {deployment_resp_id} complete!")

# Allow some time for deployment URL to get ready
time.sleep(10)

# Using deployed ML model

Let's define the local path to the dataset:

In [ ]:
import glob

path_normal_images = glob.glob("../data/Images/OK/*")
path_abnormal_images = glob.glob("../data/Images/NG/*")
#print(path_normal_images)
#print(path_abnormal_images)

path_img_ok = root_path + "/data/Images/OK/"
path_img_ko = root_path + "/data/Images/NG/"
print(path_img_ok)
print(path_img_ko)

In [ ]:
IMG_WIDTH=224
IMG_HEIGHT=224

def create_dataset(img_folder):
    img_data_array = []
    for file in os.listdir(img_folder):
            image_path = os.path.join(img_folder, file)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255
            img_data_array.append(image)
    return img_data_array

In [ ]:
img_dataset_ok = create_dataset(path_img_ok)
img_dataset_ko = create_dataset(path_img_ko)
print(len(img_dataset_ok))
print(len(img_dataset_ko))

First let's visualize an example:

In [ ]:
plt.figure(figsize = (5, 5))
plt.imshow(img_dataset_ko[0], interpolation='nearest')

In order to perform the inference step, let's transform one of the images into a string (this will constitute the body of the API call):

In [ ]:
from base64 import b64encode
import base64
import io
from json import dumps

ENCODING = 'utf-8'

# first: reading the binary stuff
# note the 'rb' flag
# result: bytes
with open(path_normal_images[0], 'rb') as open_file:
    byte_content = open_file.read()

# second: base64 encode read data
# result: bytes (again)
base64_bytes = b64encode(byte_content)

# third: decode these bytes to text
# result: string (in utf-8)
base64_string = base64_bytes.decode(ENCODING)

# optional: doing stuff with the data
# result here: some dict
raw_data = {"image": base64_string}

In [ ]:
f = open("image.txt", "w")
f.write(base64_string)
f.close()

In [ ]:
# Preparing the input for inference
# prediciton: normal product

endpoint = f"{deployment.deployment_url}/v1/models/imagemodel:predict"
print(endpoint)

headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           'ai-resource-group': resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=raw_data)

print('Inference result:', response.json())
#pprint(vars(response))

# Stop deployed model

In [ ]:
delete_resp = ai_api_v2_client.deployment.modify(deployment_resp.id,
                                                 target_status=TargetStatus.STOPPED)

#deployment_resp_id = "d835352306a37be6"
#delete_resp = ai_api_v2_client.deployment.modify(deployment_resp_id,
#                                                 target_status=TargetStatus.STOPPED)

status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    #deployment = ai_api_v2_client.deployment.get(deployment_resp_id)
    status = deployment.status
    print('...... killing deployment ......', flush=True)
    print(f"Deployment status: {deployment.status}")